In [75]:
import pandas as pd
import numpy as np

In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
# import torch.nn as nn
# from torch.utils.data import DataLoader
# import torch.optim as optim
# import torch
from sklearn.model_selection import train_test_split
import os
import sys
from sklearn.metrics import accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report


In [77]:
# Set the root directory of your repo as part of the path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [78]:
data = pd.read_csv('../Data/combined/full_lathan.csv')

/tmp/ipykernel_489336/4119657318.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/combined/full_lathan.csv')


In [80]:
# Vegas data
vegas = pd.read_csv('../Data/combined/full_vegas_data.csv')
vegas.head()

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,...,Average_Odds_OU,Best_Line_OU,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total,YEAR
0,2012-10-30,away,Washington,Cleveland,1610612764,21200001,44.6,220.0,210.0,200.0,...,-108.17,191.0,187.0,-104.0,-110.0,84.0,-10.0,L,178.0,2012-2013
1,2012-10-30,home,Cleveland,Washington,1610612739,21200001,55.4,-260.0,-235.0,-240.0,...,-110.17,191.0,187.0,-106.0,-115.0,94.0,10.0,W,178.0,2012-2013
2,2012-10-30,away,Boston,Miami,1610612738,21200002,44.8,220.0,244.0,240.0,...,-107.33,188.0,182.0,-104.0,-110.0,107.0,-13.0,L,227.0,2012-2013
3,2012-10-30,home,Miami,Boston,1610612748,21200002,55.2,-260.0,-275.0,-260.0,...,-109.33,188.0,182.0,-105.0,-115.0,120.0,13.0,W,227.0,2012-2013
4,2012-10-30,away,Dallas,L.A. Lakers,1610612742,21200003,40.8,350.0,393.0,420.0,...,-107.50,187.0,185.0,-100.0,-110.0,99.0,8.0,W,190.0,2012-2013


In [91]:
# Editing function to combine team rows of vegas data
def combine_team_rows(df):
    """
    Combines rows from the same GAME_ID into one row (1 for away, 1 for home).
    Assumes the first team listed per GAME_ID is the away team.
    Adds 'a_' and 'h_' prefixes for features accordingly.
    """
    # Check that all games have exactly 2 rows
    game_counts = df['GameId'].value_counts()
    if not (game_counts == 2).all():
        raise ValueError("Some GAME_IDs do not have exactly 2 teams.")

    # Sort the dataframe to ensure consistent ordering
    df_sorted = df.sort_values(['GameId', 'TeamId']).copy()

    # Columns to retain only once (shared columns)
    shared_cols = ['GameId', 'Date', 'YEAR']

    # Create away and home dataframes
    away_rows = df_sorted.groupby('GameId').nth(0).copy()
    home_rows = df_sorted.groupby('GameId').nth(1).copy()

    # Reset index so GAME_ID becomes a column again
    away_rows = away_rows.reset_index()
    home_rows = home_rows.reset_index()

    # Extract shared info from away team (can be either)
    shared = away_rows[shared_cols].copy()

    # Drop shared columns from both to avoid duplication
    away_features = away_rows.drop(columns=shared_cols)
    home_features = home_rows.drop(columns=shared_cols)

    # Add prefixes
    away_features = away_features.add_prefix('a_')
    home_features = home_features.add_prefix('h_')

    # Combine all into one DataFrame
    combined = pd.concat([shared, away_features, home_features], axis=1)

    return combined

In [92]:
# Creating 1 row per game for vegas data
vegas = combine_team_rows(vegas)

In [63]:
from src.data.preprocess import (
    combine_team_rows,
    add_home_win_column,
    split_wins_losses,
    add_playoff_indicator,
    combine_date_columns,
    remove_all_star_and_playoff_games,
    create_ml_dataset

)

In [64]:
# Creating key predictor columns
data2 = create_ml_dataset(data)

In [65]:
# Filling in GAME_DATE_EST Nans
data2['GAME_DATE_EST'] = data2['Date']

In [66]:
data2.columns

Index(['GAME_ID', 'GAME_SEQUENCE', 'Date', 'YEAR', 'GAME_DATE_EST', 'a_index',
       'a_TEAM_ID', 'a_TEAM_WINS_LOSSES', 'a_PTS', 'a_FG_PCT', 'a_FT_PCT',
       'a_FG3_PCT', 'a_AST', 'a_REB', 'a_TOV', 'h_index', 'h_TEAM_ID',
       'h_TEAM_WINS_LOSSES', 'h_PTS', 'h_FG_PCT', 'h_FT_PCT', 'h_FG3_PCT',
       'h_AST', 'h_REB', 'h_TOV', 'HOME_WIN', 'a_WINS', 'a_LOSSES', 'h_WINS',
       'h_LOSSES', 'a_SEASON_GAMES_PLAYED', 'h_SEASON_GAMES_PLAYED',
       'IS_PLAYOFF_GAME', 'a_season_PTS_lag', 'a_season_AST_lag',
       'a_season_REB_lag', 'a_season_TOV_lag', 'a_season_FG_PCT_avg',
       'a_season_FT_PCT_avg', 'a_season_FG3_PCT_avg', 'a_WIN_PCT',
       'a_WIN_PCT_LAST10', 'h_season_PTS_lag', 'h_season_AST_lag',
       'h_season_REB_lag', 'h_season_TOV_lag', 'h_season_FG_PCT_avg',
       'h_season_FT_PCT_avg', 'h_season_FG3_PCT_avg', 'h_WIN_PCT',
       'h_WIN_PCT_LAST10'],
      dtype='object')

### Setting up Data for the Model

In [67]:
# Adding season columns (ex: Season 2012-2013 is season 1)
season_order = sorted(data2['YEAR'].unique())
season_to_index = {season: i+1 for i, season in enumerate(season_order)}
data2['SEASON_NUM'] = data2['YEAR'].map(season_to_index)

# Dropping rows with Missing values for now
data2 = data2.dropna()

In [68]:
# Filtering data into train (2012 - 2016), validation (2017), test seasons (2018-2019)
train_df = data2[data2['SEASON_NUM'] <= 3]    # Seasons 1-3
val_df = data2[data2['SEASON_NUM'] == 4]      # Season 4
test_df = data2[data2['SEASON_NUM'] >= 5]     # Seasons 5-6

In [69]:
# Dropping columns that we don't need?
columns_to_drop = ['a_PTS', 'a_FG_PCT', 'a_FT_PCT','a_FG3_PCT', 'a_AST', 'a_REB', 'a_TOV','GAME_ID',
                   'h_PTS', 'h_FG_PCT', 'h_FT_PCT', 'h_FG3_PCT','h_AST', 'h_REB', 'h_TOV',
                   'GAME_SEQUENCE', 'Date', 'YEAR', 'a_index', 'a_TEAM_ID','h_TEAM_ID', 'a_TEAM_WINS_LOSSES',
                  'h_TEAM_WINS_LOSSES', 'IS_PLAYOFF_GAME', 'GAME_DATE_EST', 'h_index','SEASON_NUM', 'HOME_WIN']


# Creating train, validation, and test sets
X_train = train_df.drop(columns_to_drop, axis = 1)
y_train = train_df['HOME_WIN']

X_val = val_df.drop(columns_to_drop, axis = 1)
y_val = val_df['HOME_WIN']

X_test = test_df.drop(columns_to_drop, axis = 1)
y_test = test_df['HOME_WIN']

In [83]:
# Checking Nans totals for differnet columns
vegas.isna().sum().sort_values(ascending=False)

Bovada_ML                3370
Open_Line_ML              336
Betonline_ML              288
Bovada_Odds_OU            280
Bovada_Line_OU            280
Open_Odds_OU              173
Open_Line_OU              173
Heritage_ML               169
Heritage_Line_OU          153
Heritage_Odds_OU          153
Open_Line_Spread          136
Open_Odds_Spread          136
5dimes_Odds_OU            109
5dimes_Line_OU            109
Pinnacle_ML               106
5dimes_ML                  94
Betonline_Line_OU          82
Betonline_Odds_OU          82
Pinnacle_Odds_OU           78
Pinnacle_Line_OU           78
Heritage_Line_Spread       75
Heritage_Odds_Spread       75
Bovada_Line_Spread         64
Bovada_Odds_Spread         64
Pinnacle_Odds_Spread       44
Pinnacle_Line_Spread       44
Betonline_Line_Spread      22
Betonline_Odds_Spread      22
5dimes_Odds_Spread         18
5dimes_Line_Spread         18
Best_Odds_Spread            4
Worst_Odds_Spread           4
Average_Line_Spread         4
Worst_Line

### Feature Engineering:

In [71]:
# Scaling the data with mean 0 and standard deviation 1

# Fit scaler on training features only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Apply same scaler to val/test
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [73]:
# Testing TabNet Model
clf = TabNetClassifier(
    n_d=16,               # Number of decision prediction steps
    n_a=16,               # Number of attention steps (usually equal to n_d)
    n_steps=5,            # Number of steps in TabNet
    gamma=1.5,            # Relaxation factor
    lambda_sparse=1e-4,   # Sparse regularization
    optimizer_fn=torch.optim.Adam,  
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},  
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=1,
    seed=42
)

clf.fit(
    X_train=X_train_scaled, y_train=y_train,
    eval_set=[(X_val_scaled, y_val)],
    eval_name=["val"],
    eval_metric=["accuracy"],
    max_epochs=100,
    patience=10,         # Early stopping
    batch_size=1024, 
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

/home/ehe5bn/.local/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.98759 | val_accuracy: 0.53998 |  0:00:00s
epoch 1  | loss: 0.83197 | val_accuracy: 0.58697 |  0:00:00s
epoch 2  | loss: 0.74297 | val_accuracy: 0.59604 |  0:00:00s
epoch 3  | loss: 0.71295 | val_accuracy: 0.59522 |  0:00:00s
epoch 4  | loss: 0.70011 | val_accuracy: 0.58203 |  0:00:01s
epoch 5  | loss: 0.67949 | val_accuracy: 0.62077 |  0:00:01s
epoch 6  | loss: 0.66851 | val_accuracy: 0.60511 |  0:00:01s
epoch 7  | loss: 0.66235 | val_accuracy: 0.62242 |  0:00:01s
epoch 8  | loss: 0.66488 | val_accuracy: 0.59769 |  0:00:02s
epoch 9  | loss: 0.66551 | val_accuracy: 0.61088 |  0:00:02s
epoch 10 | loss: 0.65399 | val_accuracy: 0.59439 |  0:00:02s
epoch 11 | loss: 0.6501  | val_accuracy: 0.60841 |  0:00:02s
epoch 12 | loss: 0.65284 | val_accuracy: 0.60594 |  0:00:03s
epoch 13 | loss: 0.6522  | val_accuracy: 0.61006 |  0:00:03s
epoch 14 | loss: 0.64835 | val_accuracy: 0.60346 |  0:00:03s
epoch 15 | loss: 0.64568 | val_accuracy: 0.61006 |  0:00:03s
epoch 16 | loss: 0.64853

/home/ehe5bn/.local/lib/python3.11/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [74]:
# Predicting on test data
y_pred = clf.predict(X_test_scaled)
y_proba = clf.predict_proba(X_test_scaled)[:, 1]

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

Accuracy: 0.6057111477210324
ROC AUC: 0.6376477429251237
              precision    recall  f1-score   support

           0       0.60      0.63      0.61      1801
           1       0.62      0.58      0.60      1841

    accuracy                           0.61      3642
   macro avg       0.61      0.61      0.61      3642
weighted avg       0.61      0.61      0.61      3642

